In [2]:
import numpy as np
import pandas as pd

In [3]:
import urllib.request
from pathlib import Path
import os
path = Path()

# Dictionary of file names and download links
files = {'listings.csv':'http://data.insideairbnb.com/united-states/ny/new-york-city/2020-09-07/visualisations/listings.csv'}

# Download each file
for key,value in files.items():
    filename = path/key
    url = value
    # If the file does not already exist in the directory, download it
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url,filename)

In [5]:
Airbnb = pd.read_csv('listings.csv')
Airbnb.shape
Airbnb['neighbourhood_group'].unique()

array(['Manhattan', 'Brooklyn', 'Queens', 'Staten Island', 'Bronx'],
      dtype=object)

In [4]:
print(list(Airbnb.columns))

['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']


In [6]:
# Keep columns that would be useful for analysis
AirbnbInfo = Airbnb[['id','name','latitude','longitude', 'room_type', 'price', 'availability_365', 'neighbourhood_group']]

In [7]:
AirbnbInfo.head()

,id,name,latitude,longitude,room_type,price,availability_365,neighbourhood_group
0,2595,Skylit Midtown Castle,40.75362,-73.98377,Entire home/apt,175,365,Manhattan
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",40.68514,-73.95976,Entire home/apt,76,272,Brooklyn
2,5121,BlissArtsSpace!,40.68688,-73.95596,Private room,60,365,Brooklyn
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",40.66120,-73.99423,Entire home/apt,175,352,Brooklyn
4,5178,Large Furnished Room Near B'way,40.76489,-73.98493,Private room,79,346,Manhattan


In [8]:
# Drop the rows with zero availability of a year
AirbnbInfo = AirbnbInfo[AirbnbInfo['availability_365'] != 0]
AirbnbInfo = AirbnbInfo.reset_index(drop=True)
AirbnbInfo

,id,name,latitude,longitude,room_type,price,availability_365,neighbourhood_group
0,2595,Skylit Midtown Castle,40.753620,-73.983770,Entire home/apt,175,365,Manhattan
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",40.685140,-73.959760,Entire home/apt,76,272,Brooklyn
2,5121,BlissArtsSpace!,40.686880,-73.955960,Private room,60,365,Brooklyn
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",40.661200,-73.994230,Entire home/apt,175,352,Brooklyn
4,5178,Large Furnished Room Near B'way,40.764890,-73.984930,Private room,79,346,Manhattan
...,...,...,...,...,...,...,...,...
25643,45327718,Spacious 1 Queen Bedroom in Central Park,40.802660,-73.958360,Private room,53,365,Manhattan
25644,45329132,"5,000 sq ft Downtown Brooklyn Penthouse",40.687770,-73.982390,Private room,79,361,Brooklyn
25645,45329491,Spacious 1 Queen Bedroom in Tribeca,40.713630,-74.008470,Private room,70,364,Manhattan
25646,45329984,Spacious 1 Full Bedroom in Murray Hill,40.744800,-73.977500,Private room,59,365,Manhattan


In [9]:
# Store the cleaned data to a new csv file
AirbnbInfo.to_csv('Cleaned_Airbnb.csv')

In [10]:
Airbnbnew = Airbnb.copy()
Airbnbnew = Airbnbnew[Airbnbnew['availability_365'] != 0]
Airbnbnew = Airbnbnew.reset_index(drop=True)
Airbnb_price = Airbnbnew[['id','name','latitude','longitude','neighbourhood_group','neighbourhood','room_type','price']]
Airbnb_price

,id,name,latitude,longitude,neighbourhood_group,neighbourhood,room_type,price
0,2595,Skylit Midtown Castle,40.753620,-73.983770,Manhattan,Midtown,Entire home/apt,175
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",40.685140,-73.959760,Brooklyn,Clinton Hill,Entire home/apt,76
2,5121,BlissArtsSpace!,40.686880,-73.955960,Brooklyn,Bedford-Stuyvesant,Private room,60
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",40.661200,-73.994230,Brooklyn,Sunset Park,Entire home/apt,175
4,5178,Large Furnished Room Near B'way,40.764890,-73.984930,Manhattan,Hell's Kitchen,Private room,79
...,...,...,...,...,...,...,...,...
25643,45327718,Spacious 1 Queen Bedroom in Central Park,40.802660,-73.958360,Manhattan,Harlem,Private room,53
25644,45329132,"5,000 sq ft Downtown Brooklyn Penthouse",40.687770,-73.982390,Brooklyn,Boerum Hill,Private room,79
25645,45329491,Spacious 1 Queen Bedroom in Tribeca,40.713630,-74.008470,Manhattan,Tribeca,Private room,70
25646,45329984,Spacious 1 Full Bedroom in Murray Hill,40.744800,-73.977500,Manhattan,Kips Bay,Private room,59


In [11]:
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [12]:
Airbnb_price.isnull().sum()

id                     0
name                   5
latitude               0
longitude              0
neighbourhood_group    0
neighbourhood          0
room_type              0
price                  0
dtype: int64

In [13]:
AB_neighbourhood = Airbnb_price[['neighbourhood','room_type','price']].groupby(['neighbourhood','room_type']).agg(['mean','std'])
AB_neighbourhood.columns = AB_neighbourhood.columns.droplevel()
AB_neighbourhood

mean         std
neighbourhood room_type                              
Allerton      Entire home/apt  179.230769  226.431798
              Private room      57.333333   17.981472
Arden Heights Entire home/apt  145.000000   46.486557
              Private room     125.000000         NaN
Arrochar      Entire home/apt  200.666667  204.465156
...                                   ...         ...
Woodlawn      Entire home/apt  176.000000   86.007752
              Private room      61.000000   24.248711
Woodside      Entire home/apt  159.642857   67.365897
              Private room      46.532468   27.596905
              Shared room       62.500000   58.523500

[537 rows x 2 columns]

In [14]:
newAB = pd.merge(Airbnb_price, AB_neighbourhood, on=['neighbourhood','room_type'], how='outer')
newAB

,id,name,latitude,longitude,neighbourhood_group,neighbourhood,room_type,price,mean,std
0,2595,Skylit Midtown Castle,40.75362,-73.98377,Manhattan,Midtown,Entire home/apt,175,265.455944,270.919813
1,59855,One bedroom Apt. in NYC,40.75890,-73.96991,Manhattan,Midtown,Entire home/apt,250,265.455944,270.919813
2,110739,Central Nomad/Chelsea Loft Studio - metro dnst...,40.74503,-73.98876,Manhattan,Midtown,Entire home/apt,110,265.455944,270.919813
3,235951,Stylish Studio with exclusive Terrace,40.75348,-73.97065,Manhattan,Midtown,Entire home/apt,135,265.455944,270.919813
4,271694,"Easy, comfortable studio in Midtown",40.75282,-73.97315,Manhattan,Midtown,Entire home/apt,125,265.455944,270.919813
...,...,...,...,...,...,...,...,...,...,...
25643,43836149,Studio Monarch - Photography & Production Only,40.67767,-74.00740,Brooklyn,Red Hook,Shared room,120,120.000000,NaN
25644,44224775,Stunning 2-people shared close to M train,40.70224,-73.90637,Queens,Ridgewood,Shared room,20,20.000000,NaN
25645,44322911,Oakwood Room,40.56436,-74.13019,Staten Island,Oakwood,Private room,150,150.000000,NaN
25646,44491496,COMFY COZY PLACE TO CRASH,40.60785,-73.91319,Brooklyn,Mill Basin,Shared room,85,85.000000,NaN


In [15]:
newAB.isnull().sum()

id                      0
name                    5
latitude                0
longitude               0
neighbourhood_group     0
neighbourhood           0
room_type               0
price                   0
mean                    0
std                    78
dtype: int64

In [16]:
newAB['zscore']= (newAB['price']-newAB['mean'])/newAB['std']
#the null value in standard deviation is because there's one listing in that area
#and std performs over N-1
#the null value is Z is because of null value in standard deviation
#so back fill with 0 because there's only one listing
newAB['zscore'] = newAB['zscore'].fillna(0)
newAB

,id,name,latitude,longitude,neighbourhood_group,neighbourhood,room_type,price,mean,std,zscore
0,2595,Skylit Midtown Castle,40.75362,-73.98377,Manhattan,Midtown,Entire home/apt,175,265.455944,270.919813,-0.333885
1,59855,One bedroom Apt. in NYC,40.75890,-73.96991,Manhattan,Midtown,Entire home/apt,250,265.455944,270.919813,-0.057050
2,110739,Central Nomad/Chelsea Loft Studio - metro dnst...,40.74503,-73.98876,Manhattan,Midtown,Entire home/apt,110,265.455944,270.919813,-0.573808
3,235951,Stylish Studio with exclusive Terrace,40.75348,-73.97065,Manhattan,Midtown,Entire home/apt,135,265.455944,270.919813,-0.481530
4,271694,"Easy, comfortable studio in Midtown",40.75282,-73.97315,Manhattan,Midtown,Entire home/apt,125,265.455944,270.919813,-0.518441
...,...,...,...,...,...,...,...,...,...,...,...
25643,43836149,Studio Monarch - Photography & Production Only,40.67767,-74.00740,Brooklyn,Red Hook,Shared room,120,120.000000,NaN,0.000000
25644,44224775,Stunning 2-people shared close to M train,40.70224,-73.90637,Queens,Ridgewood,Shared room,20,20.000000,NaN,0.000000
25645,44322911,Oakwood Room,40.56436,-74.13019,Staten Island,Oakwood,Private room,150,150.000000,NaN,0.000000
25646,44491496,COMFY COZY PLACE TO CRASH,40.60785,-73.91319,Brooklyn,Mill Basin,Shared room,85,85.000000,NaN,0.000000


In [17]:
outlier = newAB.loc[(newAB['zscore']>3) ^ (newAB['zscore']<-3)]
outlier

,id,name,latitude,longitude,neighbourhood_group,neighbourhood,room_type,price,mean,std,zscore
65,8810949,NYC Upscale Midtown East 3BR Apt,40.74867,-73.96734,Manhattan,Midtown,Entire home/apt,1170,265.455944,270.919813,3.338789
108,12870225,Fab 3BR UN Plaza with River Views,40.75260,-73.96789,Manhattan,Midtown,Entire home/apt,1100,265.455944,270.919813,3.080410
140,14408114,Unparalleled Luxury in Midtown Manhattan,40.76455,-73.97959,Manhattan,Midtown,Entire home/apt,2500,265.455944,270.919813,8.247991
155,15347268,Modern Luxury 2 Bed/ 2 Bath apartment in Midtown!,40.75425,-73.98312,Manhattan,Midtown,Entire home/apt,2000,265.455944,270.919813,6.402426
159,15857259,5 Star Luxury 2 BR Suite Heart of Manhattan,40.76110,-73.97412,Manhattan,Midtown,Entire home/apt,1999,265.455944,270.919813,6.398735
...,...,...,...,...,...,...,...,...,...,...,...
25170,34518079,Park Avenue Suite with Large Outdoor Terrace,40.74392,-73.98638,Manhattan,Midtown,Hotel room,1286,321.797297,270.957510,3.558501
25171,34518082,Approx. 980 sqft Suite with Large Outdoor Terrace,40.74392,-73.98638,Manhattan,Midtown,Hotel room,1186,321.797297,270.957510,3.189440
25366,41348519,Attic with Queen Bed,40.81512,-73.81752,Bronx,Throgs Neck,Private room,105,52.933333,16.628146,3.131237
25395,34718633,Mott Haven Dorm CC,40.80893,-73.93111,Bronx,Port Morris,Shared room,24,24.909091,0.301511,-3.015113


In [18]:
#delete the airbnb that has outlying prices from our dataset
FinalAB = newAB.loc[(newAB['zscore']>3) ^ (newAB['zscore']<-3)]
FinalAB = FinalAB[['id','name','latitude','longitude', 'room_type', 'price']]
FinalAB.to_csv('Final_Airbnb.csv')